# Helper functions

In [1]:
import numpy as np

In [2]:
# From Xavier
def strToBits(string):
    res = []
    byte_string = string.encode('utf-8')
    for b in byte_string:
        bit_array = bin(b)[2:]
        bit_array = '00000000'[len(bit_array):] + bit_array
        res.extend(bit_array)
    return res

def stringToChannelInput(string):
    bits = np.array(strToBits(string), dtype='int64')
    return 2*bits - 1

def channelOutputToString(channelOutput):
    bits = ((channelOutput+1)/2).astype('int64').tolist()
    byte_string = ""
    for char_index in range(len(bits)//8):
        bit_list = bits[char_index*8:(char_index+1)*8]
        byte = chr(int(''.join([str(bit) for bit in bit_list]), 2))
        byte_string += byte
    return byte_string

In [3]:
# From handout
def channel(chanInput):
    chanInput = np.clip(chanInput,-1,1)
    erasedIndex = np.random.randint(3) 
    chanInput[erasedIndex:len(chanInput):3] = 0
    return chanInput + np.sqrt(10)*np.random.randn(len(chanInput))

In [4]:
def channelWoNoise(chanInput, erasedIndex=None):
    """
    Emulates communication channel without noise.
    erasedIndex can be used to specify H
    """
    chanInput = np.clip(chanInput,-1,1)
    erasedIndex = erasedIndex
    if (erasedIndex == None):
        erasedIndex = np.random.randint(3)
    chanInput[erasedIndex:len(chanInput):3] = 0
    return chanInput

In [5]:
import string
import random
import io

In [6]:
def generateTestFile(characters=80, filename="scratch"):
    """
    Function to generate a file containing an utf-8 encoded string.
    Returns the generated text.
    """
    text = ''.join(random.choices(string.ascii_letters + string.digits + string.punctuation, k=characters))
    with io.open(filename+".txt", "w", encoding='utf8') as f:
        f.write(text)
    return text

In [7]:
def readTestFile(filename="scratch"):
    """
    Function to read a text file as channel input.
    Returns the file text in the channel input format.
    """
    text = ''
    with io.open(filename+".txt", encoding='utf8') as f:
        text = f.read()
    return stringToChannelInput(text)

In [191]:
signal_set = np.array([[1, 1, 1], [1, -1, -1], [-1, 1, -1], [-1, -1, 1]])
repetitions = 10

def encodeChannelInput(chan_input):
    """
    Makes channel input ready for transmission
    """
    # map each input pair to the signal whose two first elements are said pair
    def mapBitToSignals(bits):
        if (np.array_equal(bits, [1, 1])):
            return signal_set[0]
        elif (np.array_equal(bits, [1, -1])):
            return signal_set[1]
        elif (np.array_equal(bits, [-1, 1])):
            return signal_set[2]
        elif (np.array_equal(bits, [-1, -1])):
            return signal_set[3]

    chan_input = np.split(chan_input, len(chan_input)/2)
    chan_input = np.array([mapBitToSignals(bit) for bit in chan_input])

    # repeat each signal for redundancy
    def repeat(bits):
        res = np.empty([repetitions, bits.shape[0]])
        res[:] = bits
        return res

    chan_input = np.array([repeat(bits) for bits in chan_input])

    # flatten result
    chan_input = chan_input.flatten()
    return chan_input

In [217]:
def decodeChannelOutputWoNoise(chan_output, erasedIndex=None):
    """
    Decodes channel output where H is known
    """
    # split output into repetition blocks
    chan_output = np.split(chan_output, len(chan_output)/(repetitions * len(signal_set[0])))

    # split repetition blocks into signals
    def splitRepetitionBlocks(repetition_block):
        return np.split(repetition_block, len(repetition_block)/len(signal_set[0]))

    chan_output = np.array([splitRepetitionBlocks(repetition_block) for repetition_block in chan_output])

    # decide on H -> when to compute? for each block? once?
    def decide_on_H(bits):
        return np.argmin(np.array([x**2 for x in bits]))
    
    def decide_on_H_block(repetition_block):
        return np.array([decide_on_H(bits) for bits in repetition_block])

    H = np.array([decide_on_H_block(repetition_block) for repetition_block in chan_output])
    H = H.flatten()
    erasedIndex = np.bincount(H).argmax()

    # decide on index of codeword
    def decoder_H0(input):
        if (input[1] > 0 and input[2] > 0):
            return 0
        elif (input[1] > 0 and input[2] < 0):
            return 2
        elif (input[1] < 0 and input[2] < 0):
            return 1
        elif (input[1] < 0 and input[2] > 0):
            return 3

    def decoder_H1(input):
        if (input[0] > 0 and input[2] > 0):
            return 0
        elif (input[0] > 0 and input[2] < 0):
            return 1
        elif (input[0] < 0 and input[2] < 0):
            return 2
        elif (input[0] < 0 and input[2] > 0):
            return 3

    def decoder_H2(input):
        if (input[0] > 0 and input[1] > 0):
            return 0
        elif (input[0] > 0 and input[1] < 0):
            return 1
        elif (input[0] < 0 and input[1] < 0):
            return 3
        elif (input[0] < 0 and input[1] > 0):
            return 2

    def block_decoder(repetition_block):
        res = np.array([])
        if (erasedIndex == 0):
            res = np.array([decoder_H0(block) for block in repetition_block])
        elif (erasedIndex == 1):
            res = np.array([decoder_H1(block) for block in repetition_block])
        elif (erasedIndex == 2):
            res = np.array([decoder_H2(block) for block in repetition_block])

    chan_output = np.array([block_decoder(repetition_block) for repetition_block in chan_output])

    # reconstitute channel input
    def retrieveInput(index):
        return signal_set[index][:2]

    chan_output = np.array([retrieveInput(i) for i in chan_output])
    chan_output = res.flatten()

    return channelOutputToString(chan_output)

In [218]:
text_in = generateTestFile(characters=1)
print("input:\t" + text_in)
chan_input = encodeChannelInput(readTestFile())
print(chan_input)
chan_output = channel(chan_input)
text_out = decodeChannelOutputWoNoise(chan_output)
print("output:\t" + text_out)
diff = sum(text_in[i] != text_out[i] for i in range(len(text_out)))
print("diff:\t" + str(diff))

input:	s
[-1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1.
 -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1.
 -1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1. -1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
[array([-0.07414125,  1.20634016,  1.24513254, -3.58954189,  5.86552231,
        2.84346723, -2.10940279,  0.63613656, -0.92713674, -1.31296391,
       -0.0411952 ,  1.44902178, -0.7702476 , -5.59898833, -5.25604827,
       -2.30747106,  1.43208012,  1.3044543 , -0.42200244, -0.13527636,
        0.75808815,  3.68326767,  2.91611946, -2.99746815,  1.30281996,
       -0.96316752,  3.94361483, -1.42707997,  1.26232389,  0.59764178]), array([ 0.62706116,  3.41831765,  1.59763598, -0.9645322 ,  7.737565

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()